In [15]:
import numpy as np

class GramSchmidt:
    """
    Performs the Gram-Schmidt process to orthonormalize a set of vectors.

    The Gram-Schmidt process is an algorithm for orthonormalizing a set of vectors in an inner product space.
    It produces a set of vectors that are orthogonal (perpendicular) to each other and have a norm (length) of 1.

    Attributes:
        dimension (int): The dimension of the vectors.
        vectors (np.ndarray): A NumPy array of shape (n, d) representing the input vectors,
                             where n is the number of vectors and d is the dimension.
        orthonormal_basis (list[np.ndarray], optional): A list of orthonormal vectors resulting from the process,
                                                       initialized as None.

    Raises:
        ValueError: If the dimension is less than or equal to zero, or if any vector has a dimension less
                    than the class's dimension, or if the vectors are not linearly independent.
    """

    def __init__(self, dimension, vectors):
        """
        Initializes the GramSchmidt object.

        Args:
            dimension (int): The dimension of the vectors.
            vectors (list[list]): A list of lists representing the input vectors.

        Raises:
            ValueError: If the dimension is less than or equal to zero, or if any vector has a dimension less
                        than the class's dimension, or if the vectors are not linearly independent.
        """

        if dimension <= 0:
            raise ValueError("Dimension must be greater than 0.")

        self.dimension = dimension
        self.vectors = self._validate_and_adjust_vectors(vectors)
        if not self._check_linear_independence():
            raise ValueError("The provided vectors are not linearly independent.")

        self.orthonormal_basis = None  # Initialize as None, populated in process()

    def _validate_and_adjust_vectors(self, vectors):
        """
        Validates and adjusts the input vectors.

        Ensures all vectors have the correct dimension and adjusts them if necessary:
        - If a vector has a dimension less than the class's dimension, raises a ValueError.
        - If a vector has a dimension greater than the class's dimension, prints a warning
          and truncates it to the correct dimension.

        Args:
            vectors (list[list]): A list of lists representing the input vectors.

        Returns:
            np.ndarray: A NumPy array of shape (n, d) representing the validated and adjusted vectors.
        """

        validated_vectors = []
        for i, v in enumerate(vectors):
            if len(v) < self.dimension:
                raise ValueError(f"Vector {i} has dimension less than {self.dimension}. Please fix it.")
            elif len(v) > self.dimension:
                print(f"Warning: Vector {i} has dimension greater than {self.dimension}. It will be truncated.")
                validated_vectors.append(v[:self.dimension])
            else:
                validated_vectors.append(v)

        return np.array(validated_vectors)

    def _check_linear_independence(self):
        """
        Checks if the provided vectors are linearly independent.

        Returns:
            bool: True if the vectors are linearly independent, False otherwise.
        """
        matrix = np.vstack(self.vectors)
        rank = np.linalg.matrix_rank(matrix)
        return rank == len(self.vectors)

    def process(self):
        """
        Performs the Gram-Schmidt process on the input vectors.

        The process iterates through the vectors, projecting each vector onto the previously
        computed orthonormal basis vectors and subtracting the projection to obtain a vector orthogonal
        to the existing basis. The resulting vector is normalized if its norm is not near zero.

        Returns:
            list[np.ndarray]: A list of orthonormal vectors resulting from the process.
        """

        u = []
        for v in self.vectors:
            proj = sum(np.dot(v, ui) / np.dot(ui, ui) * ui for ui in u)
            ei = v - proj
            if np.linalg.norm(ei) > 1e-10:  # Check if ei is not near zero
                u.append(ei)

        self.orthonormal_basis = [ui / np.linalg.norm(ui) for ui in u]
        return self.orthonormal_basis

    def __str__(self):
        """
        Returns a string representation of the class with the orthonormal basis.

        Returns:
            str: A string representation of the class.
        """

        formatted_basis = ["[" + ", ".join(f"{component:.4f}" for component in vector) + "]" for vector in self.orthonormal_basis]
        return "Orthonormal basis: " + ", ".join(formatted_basis)


In [7]:
# Example usage:
vectors = [[1, 1, 0], [1, 0, 1], [0, 1, 1]]
gs = GramSchmidt(3, vectors)
orthonormal_basis = gs.process()
print(gs)

Orthonormal basis: [0.7071, 0.7071, 0.0000], [0.4082, -0.4082, 0.8165], [-0.5774, 0.5774, 0.5774]


In [16]:
# Testing with 2D vectors
vectors_2d = [[1, 2], [3, 4]]
gs_2d = GramSchmidt(2, vectors_2d)
orthonormal_basis_2d = gs_2d.process()
print(gs_2d)


Orthonormal basis: [0.4472, 0.8944], [0.8944, -0.4472]


In [17]:
# Vectors with more dimensions than specified, should be truncated
vectors_excess = [[1, 2, 3], [4, 5, 6, 7]]
gs_excess = GramSchmidt(2, vectors_excess)
orthonormal_basis_excess = gs_excess.process()
print(gs_excess)


Orthonormal basis: [0.4472, 0.8944], [0.8944, -0.4472]


In [18]:
# Vectors with less dimensions than specified, should raise an error
try:
    vectors_insufficient = [[1, 2], [3]]
    gs_insufficient = GramSchmidt(3, vectors_insufficient)
    orthonormal_basis_insufficient = gs_insufficient.process()
    print(gs_insufficient)
except ValueError as e:
    print(e)


Vector 0 has dimension less than 3. Please fix it.


In [19]:
# Including a zero vector, should handle gracefully
vectors_with_zero = [[0, 0, 0], [1, 0, 1], [0, 1, 1]]
gs_with_zero = GramSchmidt(3, vectors_with_zero)
orthonormal_basis_with_zero = gs_with_zero.process()
print(gs_with_zero)


ValueError: The provided vectors are not linearly independent.

In [20]:
# Linearly dependent vectors, should raise an error if linear independence check is implemented
try:
    vectors_dependent = [[1, 1, 1], [2, 2, 2], [3, 3, 3]]
    gs_dependent = GramSchmidt(3, vectors_dependent)
    orthonormal_basis_dependent = gs_dependent.process()
    print(gs_dependent)
except ValueError as e:
    print(e)


The provided vectors are not linearly independent.


In [21]:
# Testing with 4D vectors
vectors_4d = [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
gs_4d = GramSchmidt(4, vectors_4d)
orthonormal_basis_4d = gs_4d.process()
print(gs_4d)


Orthonormal basis: [1.0000, 0.0000, 0.0000, 0.0000], [0.0000, 1.0000, 0.0000, 0.0000], [0.0000, 0.0000, 1.0000, 0.0000], [0.0000, 0.0000, 0.0000, 1.0000]
